# Tutorial: Using EvolveAdapter with Signal Processing Example

This tutorial demonstrates how to adapt the OpenEvolve signal processing example to work with GEPA's `EvolveAdapter`. The key changes required are providing a batch of training data and modifying your `evaluate` function to accept a **single data instance** and return an `EvaluationResult` for that data instance.

> **Note:** This tutorial is also available as a standalone Python script: `tutorial.py`.

## Prerequisites

1. **Clone the GEPA repository** to get the tutorial files:
   ```bash
   git clone https://github.com/gepa-ai/gepa.git
   cd gepa
   ```

2. **Install GEPA in editable mode** (required since EvolveAdapter is in a PR branch):
   ```bash
   pip install -e ".[full]"
   ```

3. **Install dependencies and input API key** using the setup cell at the start of **Step 5: Using EvolveAdapter** below.

4. **Run this notebook from:** `src/gepa/examples/evolve_adapter/signal_processing/`


   This tutorial uses example files located in `src/gepa/examples/evolve_adapter/signal_processing/`:
   - `tutorial.ipynb` (this notebook)
   - `tutorial.py` (standalone Python script)
   - `tutorial_example/` (example OpenEvolve project containing `config.yaml`, `evaluator.py`, and `initial_program.py`)
   
   The `tutorial_example/` directory contains a complete, working example that demonstrates the required changes.
   **Important:** Restart the kernel after installation to ensure packages are loaded correctly.

## Example Project: Signal Processing

We'll use the **signal processing** example from OpenEvolve that evolves real-time adaptive filtering algorithms for non-stationary time series data.

## Step 1: Key Changes Required

Here are the **two main changes** you need to make to your `evaluate` function:

### Change 1: Function Signature
**Before:**
```python
def evaluate(program_path: str) -> dict:
```

**After:**
```python
def evaluate(program_path: str, data_instance: Any) -> EvaluationResult:
```
- Add `data_instance` parameter (can be any type - tuple, dict, custom object, etc.)
- Return `EvaluationResult` instead of `dict`
- For the signal processing example, we use a tuple `(noisy_signal, clean_signal)`, but you can use whatever type best suits your project

### Change 2: Process Single Data Instance
**Before:** Generated test signals internally and looped over them

**After:** Process the single `data_instance` provided. The adapter will call your function for each instance in the batch.

## Step 2: Modifying Cascade Evaluation Functions

If your project uses cascade evaluation (stage1, stage2), you'll need to modify those functions similarly. See the code cell in **Step 5: Using EvolveAdapter** for examples.

## Step 3: Defining Training Dataset

The training dataset is a list of data instances. Each data instance can be of **any type** (dict, tuple, string, custom object, etc.) - use whatever type best suits your original OpenEvolve project setup.

For the signal processing example, each data instance is a tuple `(noisy_signal, clean_signal)`:

```python
from evaluator import generate_test_signals

test_signals = generate_test_signals(5)  # Generate 5 different test signals
trainset = [
    signal_pair  # Each is a tuple (noisy_signal, clean_signal)
    for signal_pair in test_signals
]
```

**Note:** You can use just one data instance in your trainset if that's more analogous to your original OpenEvolve project setup. For example, if your original project evaluated on one fixed signal, using a single data instance mimics that setup.

In [10]:
# Install GEPA in editable mode (run from tutorial directory)
%pip install -e "../../../../../[full]"

Obtaining file:///Users/angelahe/Desktop/gepa
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for gepa (pyproject.toml) ... done
  Created wheel for gepa: filename=gepa-0.0.24-0.editable-py3-none-any.whl size=13500 sha256=d5dc5373f37da000d6e61d2bdff1193c728aba2050e6c8c33ab45e23ae919ab0
  Stored in directory: /private/var/folders/l_/glwc3twx7yd30d436jfkk3_00000gn/T/pip-ephem-wheel-cache-d3m6lcjn/wheels/bd/67/b3/b2d0d0c35bae524180c6a7cd56189ab7e2aee9c47a79c049a9
Successfully built gepa
  Attempting uninstall: gepa
    Found existing installation: gepa 0.0.24
    Uninstalling gepa-0.0.24:
      Successfully uninstalled gepa-0.0.24

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
# Install required dependencies
%pip install gepa openevolve numpy scipy pyyaml litellm


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Run the cell below and input your API key when prompted:

In [12]:
import os

os.environ["OPENAI_API_KEY"] = input("API Key: ")

In [13]:
from pathlib import Path

from gepa import optimize
from gepa.adapters.evolve_adapter.evolve_adapter import EvolveAdapter

## Step 5: Using EvolveAdapter

Now let's set up and run the optimization. The code is split into multiple cells for clarity.

In [14]:
# Path to your modified OpenEvolve project directory
# This should contain: config.yaml, evaluator.py, initial_program.py
# For this example, we use the tutorial_example directory in the same folder as this notebook
project_path = Path("tutorial_example")

# Import signal generation function from evaluator
import importlib.util

evaluator_path = project_path / "evaluator.py"
spec = importlib.util.spec_from_file_location("evaluator", evaluator_path)
if spec is None or spec.loader is None:
    raise ImportError(f"Could not load evaluator module from {evaluator_path}")
evaluator_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(evaluator_module)
generate_test_signals = evaluator_module.generate_test_signals

In [15]:
# Create the adapter
adapter = EvolveAdapter(path=project_path)
print(f"Adapter created for project: {project_path}")

Adapter created for project: tutorial_example


In [16]:
# Define training data (signal pairs)
# Each data_instance is a tuple (noisy_signal, clean_signal)
test_signals = generate_test_signals(5)  # Generate 5 different test signals
trainset = [
    signal_pair  # Each is a tuple (noisy_signal, clean_signal)
    for signal_pair in test_signals
]

print(f"Created training dataset with {len(trainset)} signal pairs")
print(f"First signal pair: noisy_signal length={len(trainset[0][0])}, clean_signal length={len(trainset[0][1])}")

Created training dataset with 5 signal pairs
First signal pair: noisy_signal length=500, clean_signal length=500


In [17]:
# Read initial program
with open(project_path / "initial_program.py") as f:
    initial_program = f.read()

# Define seed candidate (the program to evolve)
seed_candidate = {"program": initial_program}

print(f"Loaded initial program ({len(initial_program)} characters)")

Loaded initial program (6001 characters)


In [20]:
# Run GEPA optimization
result = optimize(
    seed_candidate=seed_candidate,
    trainset=trainset,
    adapter=adapter,
    max_metric_calls=100,  # Budget for evaluation calls - adjust as needed
    display_progress_bar=True,
)

# Display results
best_score = result.val_aggregate_scores[result.best_idx]
print(f"\nBest score: {best_score:.4f}")
print(f"Best candidate index: {result.best_idx}")
print(f"Total candidates evaluated: {len(result.candidates)}")
print(f"Total metric calls: {result.total_metric_calls}")

Iteration 0: Base program full valset score: 0.39265168271364936 over 5 / 5 examples
Iteration 1: Selected program 0 score: 0.39265168271364936


Iteration 1: Proposed new text for program: """
    Enhanced version with trend preservation using a more sophisticated approach.

    This version aims to improve upon the weighted moving average by incorporating
    adaptive filtering concepts and potentially multi-scale analysis to better
    handle non-stationarity and minimize the multi-objective criteria.

    Args:
        x: Input signal (1D array of real-valued samples)
        window_size: Size of the sliding window

    Returns:
        y: Filtered output signal
    """
    if len(x) < window_size:
        raise ValueError(f"Input signal length ({len(x)}) must be >= window_size ({window_size})")

    output_length = len(x) - window_size + 1
    y = np.zeros(output_length)

    # --- Advanced Filtering Techniques ---
    # We will use a combination of techniques to address the multi-objective optimization.
    # For improved slope change minimization, lag error, tracking accuracy, and false reversal penalty,
    # a Kalman fi

Iteration 2: New subsample score 0.87746349541573 is not better than old score 1.1044409163956326, skipping
Iteration 3: Selected program 0 score: 0.39265168271364936
Iteration 3: Proposed new text for program: """
Real-Time Adaptive Signal Processing Algorithm for Non-Stationary Time Series

This algorithm implements a sliding window approach to filter volatile, non-stationary
time series data while minimizing noise and preserving signal dynamics.
It incorporates multi-objective optimization for slope change minimization,
lag error minimization, tracking accuracy, and false reversal penalty.
Advanced techniques like adaptive filtering (Kalman filters), multi-scale processing
(wavelets), and predictive enhancement (polynomial fitting) are considered.
"""
import numpy as np
from scipy import signal
from collections import deque
from scipy.signal import savgol_filter

# --- Constants for Optimization ---
# These are heuristic values and might need tuning based on specific data characteri

Iteration 3: New subsample score 1.310354316557564 is not better than old score 1.3253188024574332, skipping
Iteration 4: Selected program 0 score: 0.39265168271364936


Iteration 4: Proposed new text for program: """
    Enhanced adaptive signal processing algorithm for non-stationary time series.

    This algorithm aims to balance noise reduction with signal dynamics preservation
    by employing a multi-objective optimization approach. It considers:
    1. Slope change minimization: Reducing spurious directional reversals.
    2. Lag error minimization: Maintaining responsiveness.
    3. Tracking accuracy: Preserving genuine signal trends.
    4. False reversal penalty: Avoiding noise-induced trend changes.

    Advanced techniques considered for integration:
    - Adaptive filtering (Kalman filters, particle filters)
    - Multi-scale processing (wavelets, EMD)
    - Predictive enhancement (polynomial fitting, neural networks)
    - Trend detection methods

    Args:
        x: Input signal (1D array of real-valued samples)
        window_size: Size of the sliding window (W samples)

    Returns:
        y: Filtered output signal with length = len

...

Iteration 13: Proposed new text for program: """
Real-Time Adaptive Signal Processing Algorithm for Non-Stationary Time Series

This algorithm implements a sliding window approach to filter volatile, non-stationary
time series data while minimizing noise and preserving signal dynamics.
It aims to optimize for:
1. Slope change minimization - reducing spurious directional reversals.
2. Lag error minimization - maintaining responsiveness.
3. Tracking accuracy - preserving genuine signal trends.
4. False reversal penalty - avoiding noise-induced trend changes.

Advanced techniques are considered: adaptive filtering, multi-scale processing,
predictive enhancement, and trend detection methods.
"""
import numpy as np
from scipy import signal
from collections import deque
from scipy.signal import savgol_filter

class AdaptiveFilter:
    """
    A base class for adaptive filters, providing common functionality.
    """
    def __init__(self, window_size=20):
        self.window_size = window_si

Iteration 14: Proposed new text for program: """
Real-Time Adaptive Signal Processing Algorithm for Non-Stationary Time Series

This algorithm implements a sliding window approach to filter volatile, non-stationary
time series data while minimizing noise and preserving signal dynamics.
It incorporates advanced techniques to optimize for:
1. Slope change minimization
2. Lag error minimization
3. Tracking accuracy
4. False reversal penalty

This version introduces a Kalman filter for adaptive state estimation and a polynomial
fitting approach for trend extrapolation within the sliding window.
"""
import numpy as np
from collections import deque
from scipy.signal import savgol_filter

class KalmanFilter:
    """
    A simple Kalman Filter implementation for tracking a 1D signal.
    Assumes a constant velocity model.
    """
    def __init__(self, initial_state, initial_covariance, process_noise, measurement_noise):
        # State vector: [position, velocity]
        self.state = np.arra

Iteration 15: Proposed new text for program: """
    Real-Time Adaptive Signal Processing Algorithm for Non-Stationary Time Series

    This algorithm implements a sliding window approach to filter volatile, non-stationary
    time series data while minimizing noise and preserving signal dynamics.
    It aims to optimize for:
    1. Slope change minimization: Reducing spurious directional reversals.
    2. Lag error minimization: Maintaining responsiveness.
    3. Tracking accuracy: Preserving genuine signal trends.
    4. False reversal penalty: Avoiding noise-induced trend changes.
    """
    import numpy as np
    from scipy import signal
    from collections import deque

    # --- Advanced Filtering Techniques ---

    def kalman_filter_adaptive(x, initial_state, initial_covariance, process_noise, measurement_noise, window_size=20):
        """
        Implements an adaptive Kalman filter for real-time signal processing.

        Args:
            x: Input signal (1D array of rea

GEPA Optimization:  95%|█████████▌| 95/100 [04:42<00:14,  2.98s/rollouts]

Iteration 16: New subsample score 0.8819203957618702 is not better than old score 0.9397379105559157, skipping

Best score: 0.3927
Best candidate index: 0
Total candidates evaluated: 1
Total metric calls: 101


In [24]:
# Final evolved program
best_program_text = result.best_candidate.get("program", "")
complete_program = adapter._construct_complete_program(best_program_text)
print(complete_program)

# EVOLVE-BLOCK-START
"""
Real-Time Adaptive Signal Processing Algorithm for Non-Stationary Time Series

This algorithm implements a sliding window approach to filter volatile, non-stationary
time series data while minimizing noise and preserving signal dynamics.
"""
import numpy as np
from scipy import signal
from collections import deque


def adaptive_filter(x, window_size=20):
    """
    Adaptive signal processing algorithm using sliding window approach.

    Args:
        x: Input signal (1D array of real-valued samples)
        window_size: Size of the sliding window (W samples)

    Returns:
        y: Filtered output signal with length = len(x) - window_size + 1
    """
    if len(x) < window_size:
        raise ValueError(f"Input signal length ({len(x)}) must be >= window_size ({window_size})")

    # Initialize output array
    output_length = len(x) - window_size + 1
    y = np.zeros(output_length)

    # Simple moving average as baseline
    for i in range(output_length):
 

## Step 6: Results Comparison with OpenEvolve

Let's compare GEPA's results with the original OpenEvolve results from their signal processing example.

In [ ]:
import importlib.util
import tempfile

# Re-evaluate the best candidate to get detailed metrics
evaluator_path = project_path / "evaluator.py"
spec = importlib.util.spec_from_file_location("evaluator", evaluator_path)
if spec is None or spec.loader is None:
    raise ImportError(f"Could not load evaluator module from {evaluator_path}")
evaluator_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(evaluator_module)
evaluate = evaluator_module.evaluate

# Use the adapter's _construct_complete_program method
best_program_text = result.best_candidate.get("program", "")
complete_program = adapter._construct_complete_program(best_program_text)

with tempfile.NamedTemporaryFile(mode="w", suffix=".py", delete=False) as f:
    f.write(complete_program)
    temp_program_path = f.name

# Evaluate on first signal pair
best_result = evaluate(temp_program_path, trainset[0])
gepa_metrics = best_result.metrics

gepa_detailed = {
    "composite_score": gepa_metrics.get("composite_score", 0.0),
    "slope_changes": gepa_metrics.get("slope_changes", 0.0),
    "correlation": gepa_metrics.get("correlation", 0.0),
    "lag_error": gepa_metrics.get("lag_error", 0.0),
}

import os

os.unlink(temp_program_path)

print("GEPA Best Candidate Detailed Metrics:")
print("=" * 50)
for metric, value in gepa_detailed.items():
    print(f"{metric}: {value:.4f}")

GEPA Best Candidate Detailed Metrics:
composite_score: 0.4406
slope_changes: 55.0000
lag_error: 1.0024
correlation: 0.8563


In [26]:
# OpenEvolve metrics listed in their README.md
openevolve_metrics = {
    "composite_score": 0.3712,
    "slope_changes": 322.8,
    "correlation": 0.147,
    "lag_error": 0.914,
}
print("OpenEvolve Best Candidate Detailed Metrics:")
print("=" * 50)
for metric, value in openevolve_metrics.items():
    print(f"{metric}: {value:.4f}")

OpenEvolve Best Candidate Detailed Metrics:
composite_score: 0.3712
slope_changes: 322.8000
correlation: 0.1470
lag_error: 0.9140
